In [3]:
# Python ≥3.5 is required
import sys

import keras.datasets.mnist

print("Python: ", sys.version_info)
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.8 is required
import tensorflow as tf
print("TF version: ", tf.__version__)
# assert tf.__version__ >= "2.8"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow_datasets.core.utils import gcs_utils
gcs_utils._is_gcs_disabled = True


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

Python:  sys.version_info(major=3, minor=9, micro=19, releaselevel='final', serial=0)
sklearn version:  1.4.1.post1
TF version:  2.15.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


In [4]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "deep"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    savefig = plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
! pip install --upgrade tfds-nightly

## get the data(EMNIST)

In [15]:
import tensorflow_datasets as tfds

(train_data, valid_data, test_data), info = tfds.load("emnist/byclass", split=["train[:80%]", "train[80%:]", "test"], as_supervised=True, with_info=True)

In [16]:
print(f"size of train: {train_data.cardinality().numpy()}")
print(f"size of validation: {valid_data.cardinality().numpy()}")
print(f"size of test: {test_data.cardinality().numpy()}")

size of train: 558346
size of validation: 139586
size of test: 116323


In [17]:
num_class = info.features["label"].num_classes
print(num_class)

62


## prepare the data

In [18]:
def preprocess_resnet50(image, label):
    # Resize image to (224, 224, 1)
    image = tf.image.resize(image, size=(224, 224))
    # Convert grayscale to RGB by duplicating the single channel
    image = tf.image.grayscale_to_rgb(image)
    # Rescale pixel values to [0, 1]
    image = tf.cast(image, tf.float64) / 255.0
    return image, label

In [19]:
train_data = train_data.map(preprocess_resnet50)
valid_data = valid_data.map(preprocess_resnet50)
test_data = test_data.map(preprocess_resnet50)

In [20]:
batch_size = 32
train_data = train_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

## train / evaluate ResNet50

In [25]:
resNet = tf.keras.applications.ResNet50(classes=num_class, weights=None)
resNetv2 = tf.keras.applications.ResNet50V2(classes=num_class, weights=None)

resNet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

[transfer learning](https://keras.io/guides/transfer_learning/) 참고

In [27]:
resNet.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

In [29]:
history = resNet.fit(train_data, epochs=5, validation_data=valid_data, batch_size=1000)

Epoch 1/5
  441/17449 [..............................] - ETA: 44:17 - loss: 0.8834 - accuracy: 0.7321

KeyboardInterrupt: 

(224,224,3) 으로 진행하니 epoch 당 40분이 걸림...

# TODO:
1. callback 추가(earlystopping 등)
2. tensorboard로 model training 시각화(교재 참고) 
3. 지금은 (224,224,3) 으로 resize 해서 넣었는데, 매우 매우 비효율적이며 원본 이미지 역시 손상되는 방식으로 진행됨. 
4. ResNet의 훈련 방식으로
> 이미 있는 weight에서 일부 layer freeze 후 진행
> 그냥 새로 진행
이 있는데, 